### Import Libraries

In [1]:
import lseg.data as ld
import lseg.data.delivery.cfs as cfs
from datetime import timedelta

### Open the Session

In [2]:
ld.open_session('platform.ldp')
#ld.open_session(config_name='./lseg-data.devrel.config.json')

<lseg.data.session.Definition object at 0x11657bcb0 {name='ldp'}>

In [ ]:
package_id = 'Your Package ID'
bucket_name = 'bulk-Custom'

### Get the File Set information from the Bucket Name and Package ID

List the 100 file sets of this bucket name and package id

In [4]:
response = cfs.file_sets.Definition(
    bucket = bucket_name, 
    package_id = package_id,
    page_size = 100).get_data()
response.data.df

,numFiles,packageId,availableFrom,files,id,name,attributes,contentFrom,created,contentTo,modified,status,bucketName,availableTo
0,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-10T18:35:58Z,[456c-388a-1e9b601a-afff-03fb592bcd20],4006-9157-7a0fc5e1-862e-950478c22a4e,TM3_SIFMAIndex2025-06-10T14:35:00.000-04:00,[],2025-06-10T18:30:00Z,2025-06-10T18:35:58Z,2025-06-10T18:35:00Z,2025-06-10T18:36:01Z,READY,bulk-custom,2025-12-10T18:35:58Z
1,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-06T17:27:36Z,[4ce5-f175-af239ab4-901e-980e8421fa5b],400a-5326-23b3c2bf-87ab-e944e06ac777,TM3_SIFMAIndex2025-05-06T13:25:00.000-04:00,[],2025-05-06T17:20:00Z,2025-05-06T17:27:36Z,2025-05-06T17:25:00Z,2025-05-06T17:27:39Z,READY,bulk-custom,2025-11-06T17:27:36Z
2,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-22T18:11:27Z,[43d7-c6a9-5606cc46-aadb-3a2956aee17e],400a-b124-8d5b285c-b8e7-196c8cb6b363,TM3_SIFMAIndex2025-05-22T14:10:00.000-04:00,[],2025-05-22T18:05:00Z,2025-05-22T18:11:27Z,2025-05-22T18:10:00Z,2025-05-22T18:11:30Z,READY,bulk-custom,2025-11-22T18:11:27Z
3,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-06T20:42:00Z,[4122-202a-76877d41-a207-294439244348],4011-3ee5-761d8137-b58b-ae98c9f1e96b,TM3_SIFMAIndex2025-05-06T16:40:00.000-04:00,[],2025-05-06T20:35:00Z,2025-05-06T20:42:00Z,2025-05-06T20:40:00Z,2025-05-06T20:42:02Z,READY,bulk-custom,2025-11-06T20:41:59Z
4,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-10T20:06:16Z,[481e-01d3-ab6b0052-af23-98bbef9e5f1d],4011-95bc-815fe9b1-bbcd-e3459880c841,TM3_SIFMAIndex2025-06-10T16:05:00.000-04:00,[],2025-06-10T20:00:00Z,2025-06-10T20:06:16Z,2025-06-10T20:05:00Z,2025-06-10T20:06:19Z,READY,bulk-custom,2025-12-10T20:06:16Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-04-24T16:11:29Z,[49bb-d8ab-73518718-ba96-eddbb1f2618f],40cb-3630-fd6a8b1b-8c55-10ae3d2ba00f,TM3_SIFMAIndex2025-04-24T12:10:00.000-04:00,[],2025-04-24T16:05:00Z,2025-04-24T16:11:29Z,2025-04-24T16:10:00Z,2025-04-24T16:11:31Z,READY,bulk-custom,2025-10-24T16:11:28Z
96,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-17T18:16:39Z,[4d16-1dfa-7a4479bb-b09c-528b517248dc],40ce-cb00-4fb5e61d-b7c4-b64c5eeffe7d,TM3_SIFMAIndex2025-06-17T14:15:00.000-04:00,[],2025-06-17T18:10:00Z,2025-06-17T18:16:39Z,2025-06-17T18:15:00Z,2025-06-17T18:16:42Z,READY,bulk-custom,2025-12-17T18:16:39Z
97,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-14T19:41:24Z,[46d2-cdb7-74152835-92aa-7b63d35e350c],40cf-03d0-6b5ddbe5-9a66-f5974b635198,TM3_SIFMAIndex2025-05-14T15:40:00.000-04:00,[],2025-05-14T19:35:00Z,2025-05-14T19:41:24Z,2025-05-14T19:40:00Z,2025-05-14T19:41:27Z,READY,bulk-custom,2025-11-14T19:41:24Z
98,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-10T20:41:17Z,[41c7-46e1-fa35cc97-884d-980b11395e84],40d0-d848-813d58d6-a6d1-05519f086db7,TM3_SIFMAIndex2025-06-10T16:40:00.000-04:00,[],2025-06-10T20:35:00Z,2025-06-10T20:41:17Z,2025-06-10T20:40:00Z,2025-06-10T20:41:20Z,READY,bulk-custom,2025-12-10T20:41:17Z


### Get FileSets with more filters

In [5]:
response = cfs.file_sets.Definition(
    bucket = bucket_name, 
    package_id = package_id,
    page_size = 100,
    modified_since="2025-07-25T12:00:00Z"
    ).get_data()
response.data.df

,numFiles,packageId,availableFrom,files,id,name,attributes,contentFrom,created,contentTo,modified,status,bucketName,availableTo
0,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-07-29T20:56:38Z,[4b7d-cd53-0631a812-9207-95b66f8fc5ca],4af1-0efc-df11c8a3-b0ce-6d0a70b6f0e9,TM3_SIFMAIndex2025-07-29,[],2025-07-29T20:50:00Z,2025-07-29T20:56:38Z,2025-07-29T20:55:00Z,2025-07-29T20:56:41Z,READY,bulk-custom,2026-01-29T20:56:37Z
1,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-07-30T20:56:33Z,[4eda-d4b4-34cf5dab-a6d9-e9a8ed3031fa],4460-b76d-fac7ad8c-ba51-020f300f8bf7,TM3_SIFMAIndex2025-07-30,[],2025-07-30T20:50:00Z,2025-07-30T20:56:33Z,2025-07-30T20:55:00Z,2025-07-30T20:56:36Z,READY,bulk-custom,2026-01-30T20:56:33Z
2,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-07-31T20:56:14Z,[4e6a-aad8-032daa50-8f36-1a0c1af4943c],4a41-3199-7401bee9-93a6-98414cb72b5c,TM3_SIFMAIndex2025-07-31,[],2025-07-31T20:50:00Z,2025-07-31T20:56:14Z,2025-07-31T20:55:00Z,2025-07-31T20:56:16Z,READY,bulk-custom,2026-01-31T20:56:13Z
3,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-08-05T20:56:18Z,[4ec8-010a-3379a1f2-86d9-14631e49e74b],4cf8-0b9d-e47fec9d-815e-617500a3ed36,TM3_SIFMAIndex2025-08-05,[],2025-08-05T20:50:00Z,2025-08-05T20:56:18Z,2025-08-05T20:55:00Z,2025-08-05T20:56:21Z,READY,bulk-custom,2026-02-05T20:56:17Z
4,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-08-06T20:56:09Z,[4229-c5f0-6231c0e5-8499-62fe598d63ae],4330-85b2-cf675f1f-90e2-2650ee959341,TM3_SIFMAIndex2025-08-06,[],2025-08-06T20:50:00Z,2025-08-06T20:56:09Z,2025-08-06T20:55:00Z,2025-08-06T20:56:11Z,READY,bulk-custom,2026-02-06T20:56:08Z
5,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-08-07T20:56:07Z,[4892-55ea-56b0a98a-ac80-c24d16ac34f0],4b20-1b16-c921a7b1-b824-6443c8f3727a,TM3_SIFMAIndex2025-08-07,[],2025-08-07T20:50:00Z,2025-08-07T20:56:07Z,2025-08-07T20:55:00Z,2025-08-07T20:56:10Z,READY,bulk-custom,2026-02-07T20:56:06Z
6,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-08-12T20:56:21Z,[45b7-c5b9-dc845e53-82d9-2313307fb8c6],470e-2420-357307ff-98da-2b1da226939f,TM3_SIFMAIndex2025-08-12,[],2025-08-12T20:50:00Z,2025-08-12T20:56:21Z,2025-08-12T20:55:00Z,2025-08-12T20:56:24Z,READY,bulk-custom,2026-02-12T20:56:20Z
7,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-08-13T20:56:28Z,[42ae-f644-4f7439b3-ae19-224d6af53e17],486e-565f-70e9e3f9-95ad-15cd2b8a1c3a,TM3_SIFMAIndex2025-08-13,[],2025-08-13T20:50:00Z,2025-08-13T20:56:28Z,2025-08-13T20:55:00Z,2025-08-13T20:56:31Z,READY,bulk-custom,2026-02-13T20:56:27Z


#### The FileSets can be more than 100!!

Please note that the FileSets can be more than 100 records. The API returns data maximum 100 records per one query.

You can do the paging to turn on the next page as follows:

In [6]:
page1_response = cfs.file_sets.Definition(
    bucket = bucket_name, 
    package_id = package_id,
    page_size = 100).get_data()
display(page1_response.data.df)


token = page1_response.data.raw['skip_token']

page2_response = cfs.file_sets.Definition(
    bucket='bulk-Custom',
    page_size=100,
    skip_token=token,
).get_data()
display(page2_response.data.df)

,numFiles,packageId,availableFrom,files,id,name,attributes,contentFrom,created,contentTo,modified,status,bucketName,availableTo
0,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-10T18:35:58Z,[456c-388a-1e9b601a-afff-03fb592bcd20],4006-9157-7a0fc5e1-862e-950478c22a4e,TM3_SIFMAIndex2025-06-10T14:35:00.000-04:00,[],2025-06-10T18:30:00Z,2025-06-10T18:35:58Z,2025-06-10T18:35:00Z,2025-06-10T18:36:01Z,READY,bulk-custom,2025-12-10T18:35:58Z
1,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-06T17:27:36Z,[4ce5-f175-af239ab4-901e-980e8421fa5b],400a-5326-23b3c2bf-87ab-e944e06ac777,TM3_SIFMAIndex2025-05-06T13:25:00.000-04:00,[],2025-05-06T17:20:00Z,2025-05-06T17:27:36Z,2025-05-06T17:25:00Z,2025-05-06T17:27:39Z,READY,bulk-custom,2025-11-06T17:27:36Z
2,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-22T18:11:27Z,[43d7-c6a9-5606cc46-aadb-3a2956aee17e],400a-b124-8d5b285c-b8e7-196c8cb6b363,TM3_SIFMAIndex2025-05-22T14:10:00.000-04:00,[],2025-05-22T18:05:00Z,2025-05-22T18:11:27Z,2025-05-22T18:10:00Z,2025-05-22T18:11:30Z,READY,bulk-custom,2025-11-22T18:11:27Z
3,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-06T20:42:00Z,[4122-202a-76877d41-a207-294439244348],4011-3ee5-761d8137-b58b-ae98c9f1e96b,TM3_SIFMAIndex2025-05-06T16:40:00.000-04:00,[],2025-05-06T20:35:00Z,2025-05-06T20:42:00Z,2025-05-06T20:40:00Z,2025-05-06T20:42:02Z,READY,bulk-custom,2025-11-06T20:41:59Z
4,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-10T20:06:16Z,[481e-01d3-ab6b0052-af23-98bbef9e5f1d],4011-95bc-815fe9b1-bbcd-e3459880c841,TM3_SIFMAIndex2025-06-10T16:05:00.000-04:00,[],2025-06-10T20:00:00Z,2025-06-10T20:06:16Z,2025-06-10T20:05:00Z,2025-06-10T20:06:19Z,READY,bulk-custom,2025-12-10T20:06:16Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-04-24T16:11:29Z,[49bb-d8ab-73518718-ba96-eddbb1f2618f],40cb-3630-fd6a8b1b-8c55-10ae3d2ba00f,TM3_SIFMAIndex2025-04-24T12:10:00.000-04:00,[],2025-04-24T16:05:00Z,2025-04-24T16:11:29Z,2025-04-24T16:10:00Z,2025-04-24T16:11:31Z,READY,bulk-custom,2025-10-24T16:11:28Z
96,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-17T18:16:39Z,[4d16-1dfa-7a4479bb-b09c-528b517248dc],40ce-cb00-4fb5e61d-b7c4-b64c5eeffe7d,TM3_SIFMAIndex2025-06-17T14:15:00.000-04:00,[],2025-06-17T18:10:00Z,2025-06-17T18:16:39Z,2025-06-17T18:15:00Z,2025-06-17T18:16:42Z,READY,bulk-custom,2025-12-17T18:16:39Z
97,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-05-14T19:41:24Z,[46d2-cdb7-74152835-92aa-7b63d35e350c],40cf-03d0-6b5ddbe5-9a66-f5974b635198,TM3_SIFMAIndex2025-05-14T15:40:00.000-04:00,[],2025-05-14T19:35:00Z,2025-05-14T19:41:24Z,2025-05-14T19:40:00Z,2025-05-14T19:41:27Z,READY,bulk-custom,2025-11-14T19:41:24Z
98,1,4a2f-a444-b1f4a145-b0f9-4509ed361868,2025-06-10T20:41:17Z,[41c7-46e1-fa35cc97-884d-980b11395e84],40d0-d848-813d58d6-a6d1-05519f086db7,TM3_SIFMAIndex2025-06-10T16:40:00.000-04:00,[],2025-06-10T20:35:00Z,2025-06-10T20:41:17Z,2025-06-10T20:40:00Z,2025-06-10T20:41:20Z,READY,bulk-custom,2025-12-10T20:41:17Z


,numFiles,packageId,availableFrom,files,id,name,attributes,contentFrom,created,contentTo,modified,status,bucketName,availableTo
0,1,430f-22fc-649730f7-9fa5-23da0f062f0d,2025-04-28T13:48:05Z,[41f1-56a1-deec20cc-9741-cfc07c837143],40d4-5b61-fdaf7157-b063-da3aaee6e11b,ScaleRead2025-04-28T09:46:00.000-04:00,[],2025-04-28T13:43:00Z,2025-04-28T13:48:05Z,2025-04-28T13:46:00Z,2025-04-28T13:48:07Z,READY,bulk-custom,2025-10-28T13:48:04Z
1,1,430f-22fc-649730f7-9fa5-23da0f062f0d,2025-05-22T12:38:14Z,[49e8-b9bd-72d8a421-8a26-0886bee66f77],40d4-5f1a-1739e5b2-9316-04af7da18455,ScaleRead2025-05-22T08:37:00.000-04:00,[],2025-05-22T12:34:00Z,2025-05-22T12:38:14Z,2025-05-22T12:37:00Z,2025-05-22T12:38:17Z,READY,bulk-custom,2025-11-22T12:38:14Z
2,1,430f-22fc-649730f7-9fa5-23da0f062f0d,2025-06-27T20:38:10Z,[4bfb-eaf6-50fa3db6-862c-0b215b91c831],40d4-66ac-95a1304e-8efd-c70bc6e30a11,ScaleRead2025-06-27T16:37:00.000-04:00,[],2025-06-27T20:34:00Z,2025-06-27T20:38:10Z,2025-06-27T20:37:00Z,2025-06-27T20:38:13Z,READY,bulk-custom,2025-12-27T20:38:09Z
3,1,446f-65fc-21ce334a-b407-66dff6b4d228,2025-07-31T21:13:42Z,[42e1-3602-dcf1282a-99b2-34379145be4e],40d4-6835-14ffb81a-94db-9864cfecf57f,TM3_RGAMMD2025-07-31T17:12:00.000-04:00,[],2025-07-31T21:00:00Z,2025-07-31T21:13:42Z,2025-07-31T21:12:00Z,2025-07-31T21:13:45Z,READY,bulk-custom,2026-01-31T21:13:42Z
4,1,41a7-f70f-acb43762-b665-082bdbde825f,2025-07-29T20:01:55Z,[4e28-6e54-f778c719-bf26-635798cb4b27],40d4-8082-23ee023e-9479-94561bfee0e0,TM3_RBCMMDYields2025-07-29T16:00:00.000-04:00,[],2025-07-28T20:00:00Z,2025-07-29T20:01:55Z,2025-07-29T20:00:00Z,2025-07-29T20:01:58Z,READY,bulk-custom,2026-01-29T20:01:55Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,4384-5aaf-4595af2f-b430-2e76cb00822a,2025-07-29T20:25:36Z,[4fba-8178-a95c28d0-981e-a95b117c69f3],40db-84e6-28c3591c-9e53-2c8b851af4b4,TM3_DeutscheMMDFeed2025-07-29T16:24:00.000-04:00,[],2025-07-29T20:21:00Z,2025-07-29T20:25:36Z,2025-07-29T20:24:00Z,2025-07-29T20:25:39Z,READY,bulk-custom,2026-01-29T20:25:36Z
96,1,4264-4e82-3f59f507-bfb2-7ed62156c417,2025-07-31T12:18:09Z,[4db9-c15d-ffac1419-88bb-51d573015fee],40db-8c10-78d284a0-a46d-134de7e1b022,TM3_VRDUSBank2025-07-31T08:15:00.000-04:00,[],1970-01-01T00:00:00Z,2025-07-31T12:18:09Z,2025-07-31T12:15:00Z,2025-07-31T12:18:11Z,READY,bulk-custom,2026-01-31T12:18:09Z
97,1,47fb-8a54-e0e982eb-bff1-356de0470608,2025-05-29T21:01:51Z,[4e50-e44a-3d92bb9c-83c5-6118d5ec136a],40db-930e-7f97c39a-87a8-750de14e59f5,TM3_BOAMMDRiskMMDFeed2025-05-29T17:00:00.000-0...,[],2025-05-28T21:00:00Z,2025-05-29T21:01:51Z,2025-05-29T21:00:00Z,2025-05-29T21:01:54Z,READY,bulk-custom,2025-11-29T21:01:51Z
98,1,4b97-c1a2-09de6766-b13e-3b6fe8c481e9,2025-06-16T20:03:08Z,[4266-d323-f3ec5354-bb72-eadecc6b75f5],40db-abaf-b8e39272-ac73-a1dcc0a9451f,TM3_RBCMMDTaxable2025-06-16T16:00:00.000-04:00,[],2025-06-13T20:00:00Z,2025-06-16T20:03:08Z,2025-06-16T20:00:00Z,2025-06-16T20:03:11Z,READY,bulk-custom,2025-12-16T20:03:08Z


### Get the file URL detail from the File Set

The ```file set``` information is in the ```id``` column.
Example with the first one.

In [7]:
first_file_set_id = response.data.df.at[0,'id']

response_fileset = cfs.files.Definition(first_file_set_id).get_data()
response_fileset.data.df

,md5,filename,filesetId,fileSizeInBytes,id,storageLocation,created,modified,href
0,eeef3e416689c2405c8c1a6d4c36bd6f,TM3_SIFMAIndex2025-07-29,4af1-0efc-df11c8a3-b0ce-6d0a70b6f0e9,180,4b7d-cd53-0631a812-9207-95b66f8fc5ca,{'url': 'https://a206464-bulk-custom.s3.amazon...,2025-07-29T20:56:41Z,2025-07-29T20:56:41Z,https://api.refinitiv.com/file-store/v1/files/...


Get the actual URL detail.

In [8]:
file_url = response_fileset.data.files[0]
print(file_url)

CFSFile({'id': '4b7d-cd53-0631a812-9207-95b66f8fc5ca', 'filename': 'TM3_SIFMAIndex2025-07-29', 'fileset_id': '4af1-0efc-df11c8a3-b0ce-6d0a70b6f0e9', 'storage_location': {'url': 'https://a206464-bulk-custom.s3.amazonaws.com/GE-11328/2025/07/29/TM3_SIFMAIndex2025-07-29', '@type': 's3'}, 'created': '2025-07-29T20:56:41Z', 'modified': '2025-07-29T20:56:41Z', 'href': 'https://api.refinitiv.com/file-store/v1/files/4b7d-cd53-0631a812-9207-95b66f8fc5ca/stream', 'file_size_in_bytes': 180, 'md5': 'eeef3e416689c2405c8c1a6d4c36bd6f'})


### Download the file

Our beloved Zscaler blocks my download :p.

In [9]:
file_downloader = cfs.file_downloader.Definition(file_url).retrieve()
file_downloader.download()
file_downloader.extract()

'TM3_SIFMAIndex2025-07-29'

### Close the Session

In [10]:
ld.close_session()